In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import string
import re
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm.notebook import tqdm
import transformers
import torch
import torch.nn as nn
from transformers import get_linear_schedule_with_warmup, AdamW, get_cosine_schedule_with_warmup
from sklearn.metrics import mean_squared_error
import torch.nn.functional as F

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('../input/clrp-distilbert-training/model_tokenizer')

In [ ]:
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
test.head()

In [ ]:
MAX_LEN = 256

In [ ]:
class test_datasets():
    def __init__(self, excerpt):
        self.excerpt = excerpt
        self.maxlen = MAX_LEN
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self, index):
        excerpt = str(self.excerpt[index])
        
        tokenizer_dict = tokenizer(excerpt,
                                  return_tensors='pt',
                                  max_length=self.maxlen,
                                  padding='max_length',
                                  truncation=True)
        
        ids = tokenizer_dict['input_ids']
        mask = tokenizer_dict['attention_mask']
        
        return {'ids': torch.tensor(ids, dtype=torch.long),
               'mask': torch.tensor(mask, dtype=torch.long)}

In [ ]:
def test_function(dataloader, model, device):
    model.to(device)
    model.eval()
    predictions = []
    iterator = tqdm(enumerate(dataloader),
                   total=len(dataloader))
    for index, data in iterator:
        ids = data['ids']
        mask = data['mask']
        
        ids = ids.squeeze()
        mask = mask.squeeze()
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        
        with torch.no_grad():
            outputs = model(ids,
                           mask)
        
        outputs = outputs.logits.squeeze().detach().cpu().numpy().tolist()
        
        predictions.extend(outputs)
    
    return np.array(predictions)

In [ ]:
# model = transformers.DistilBertForSequenceClassification('../input/clrp-distilbert-training')
test_dataset = test_datasets(test.excerpt)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                             batch_size=8)
device = torch.device("cuda")
model1 = transformers.DistilBertForSequenceClassification.from_pretrained('../input/clrp-distilbert-training/model_1', num_labels=1)
model2 = transformers.DistilBertForSequenceClassification.from_pretrained('../input/clrp-distilbert-training/model_2', num_labels=1)
model3 = transformers.DistilBertForSequenceClassification.from_pretrained('../input/clrp-distilbert-training/model_3', num_labels=1)
model4 = transformers.DistilBertForSequenceClassification.from_pretrained('../input/clrp-distilbert-training/model_4', num_labels=1)
model5 = transformers.DistilBertForSequenceClassification.from_pretrained('../input/clrp-distilbert-training/model_5', num_labels=1)

preds1 = test_function(test_dataloader, model1, device)
preds2 = test_function(test_dataloader, model2, device)
preds3 = test_function(test_dataloader, model3, device)
preds4 = test_function(test_dataloader, model4, device)
preds5 = test_function(test_dataloader, model5, device)

In [ ]:
final_preds = (preds1 + preds2 + preds3 + preds4 + preds5) / 5

In [ ]:
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
submission.target = final_preds
submission.to_csv("submission.csv", index=False)

In [ ]:
final_preds